# Posture Estimation - Body Language Analysis

This notebook implements posture analysis using MediaPipe Pose.

## Goals
1. Set up MediaPipe Pose for upper body landmarks
2. Calculate shoulder alignment and stability
3. Detect fidgeting and leaning
4. Create posture scoring function
5. Export utility functions for backend integration

In [ ]:
# Install dependencies if needed
# !pip install mediapipe opencv-python numpy matplotlib

In [ ]:
import os
import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path(os.getcwd()).parent.parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
from collections import deque

print(f"OpenCV version: {cv2.__version__}")
print(f"MediaPipe version: {mp.__version__}")
print(f"Project root: {PROJECT_ROOT}")

## 1. Initialize MediaPipe Pose

In [ ]:
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Pose detector
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,  # 0=lite, 1=full, 2=heavy
    smooth_landmarks=True,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

print("MediaPipe Pose initialized")
print("33 pose landmarks (full body)")

## 2. Key Pose Landmarks

In [ ]:
# MediaPipe Pose landmark indices (relevant for upper body)
POSE_LANDMARKS = {
    'nose': 0,
    'left_eye_inner': 1,
    'left_eye': 2,
    'left_eye_outer': 3,
    'right_eye_inner': 4,
    'right_eye': 5,
    'right_eye_outer': 6,
    'left_ear': 7,
    'right_ear': 8,
    'mouth_left': 9,
    'mouth_right': 10,
    'left_shoulder': 11,
    'right_shoulder': 12,
    'left_elbow': 13,
    'right_elbow': 14,
    'left_wrist': 15,
    'right_wrist': 16,
    'left_hip': 23,
    'right_hip': 24,
}

# Upper body landmarks for posture analysis
UPPER_BODY_INDICES = [0, 11, 12, 13, 14, 15, 16, 23, 24]

print("\nKey landmarks for posture:")
print("  - Shoulders (11, 12): Alignment and level")
print("  - Nose (0): Head position")
print("  - Hips (23, 24): Body stability")

## 3. Posture Metrics

In [ ]:
def get_landmark(landmarks, idx, img_width, img_height):
    """
    Get landmark coordinates.
    
    Returns:
        (x, y, z, visibility) tuple
    """
    lm = landmarks.landmark[idx]
    return (
        lm.x * img_width,
        lm.y * img_height,
        lm.z,
        lm.visibility
    )


def calculate_shoulder_metrics(landmarks, img_width, img_height):
    """
    Calculate shoulder alignment metrics.
    
    Returns:
        dict: Shoulder metrics
    """
    left_shoulder = get_landmark(landmarks, 11, img_width, img_height)
    right_shoulder = get_landmark(landmarks, 12, img_width, img_height)
    
    # Shoulder level (y difference)
    shoulder_level_diff = abs(left_shoulder[1] - right_shoulder[1])
    
    # Shoulder width (x difference)
    shoulder_width = abs(left_shoulder[0] - right_shoulder[0])
    
    # Shoulder tilt angle (degrees)
    if shoulder_width > 0:
        tilt_angle = np.degrees(np.arctan(shoulder_level_diff / shoulder_width))
    else:
        tilt_angle = 0
    
    # Shoulder center
    center_x = (left_shoulder[0] + right_shoulder[0]) / 2
    center_y = (left_shoulder[1] + right_shoulder[1]) / 2
    
    return {
        'left_shoulder': left_shoulder[:2],
        'right_shoulder': right_shoulder[:2],
        'center': (center_x, center_y),
        'width': shoulder_width,
        'level_diff': shoulder_level_diff,
        'tilt_angle': tilt_angle,
        'is_level': tilt_angle < 5  # Less than 5 degrees is considered level
    }


def calculate_body_lean(landmarks, img_width, img_height):
    """
    Calculate body lean from shoulder and hip alignment.
    
    Returns:
        dict: Lean metrics
    """
    left_shoulder = get_landmark(landmarks, 11, img_width, img_height)
    right_shoulder = get_landmark(landmarks, 12, img_width, img_height)
    left_hip = get_landmark(landmarks, 23, img_width, img_height)
    right_hip = get_landmark(landmarks, 24, img_width, img_height)
    
    # Shoulder center
    shoulder_center_x = (left_shoulder[0] + right_shoulder[0]) / 2
    
    # Hip center
    hip_center_x = (left_hip[0] + right_hip[0]) / 2
    hip_center_y = (left_hip[1] + right_hip[1]) / 2
    shoulder_center_y = (left_shoulder[1] + right_shoulder[1]) / 2
    
    # Horizontal lean (shoulder relative to hip)
    horizontal_lean = shoulder_center_x - hip_center_x
    
    # Vertical distance (for normalization)
    vertical_distance = abs(hip_center_y - shoulder_center_y)
    
    # Lean angle
    if vertical_distance > 0:
        lean_angle = np.degrees(np.arctan(horizontal_lean / vertical_distance))
    else:
        lean_angle = 0
    
    # Determine lean direction
    if abs(lean_angle) < 3:
        lean_direction = 'upright'
    elif lean_angle > 0:
        lean_direction = 'leaning_right'
    else:
        lean_direction = 'leaning_left'
    
    return {
        'lean_angle': lean_angle,
        'lean_direction': lean_direction,
        'is_upright': abs(lean_angle) < 5
    }

print("Posture metric functions defined")

## 4. Stability Tracking

In [ ]:
class PostureTracker:
    """
    Tracks posture over time to calculate stability.
    """
    
    def __init__(self, history_size=30, fps=30):
        self.history_size = history_size
        self.fps = fps
        self.shoulder_center_history = deque(maxlen=history_size)
        self.shoulder_tilt_history = deque(maxlen=history_size)
        self.lean_angle_history = deque(maxlen=history_size)
    
    def update(self, shoulder_center, shoulder_tilt, lean_angle):
        """
        Update tracking history.
        """
        self.shoulder_center_history.append(shoulder_center)
        self.shoulder_tilt_history.append(shoulder_tilt)
        self.lean_angle_history.append(lean_angle)
    
    def calculate_stability(self):
        """
        Calculate stability from historical data.
        
        Returns:
            dict: Stability metrics
        """
        # Filter valid positions
        valid_centers = [p for p in self.shoulder_center_history if p is not None]
        valid_tilts = [t for t in self.shoulder_tilt_history if t is not None]
        valid_leans = [l for l in self.lean_angle_history if l is not None]
        
        # Position stability (how much did shoulders move?)
        if len(valid_centers) >= 2:
            x_coords = [c[0] for c in valid_centers]
            y_coords = [c[1] for c in valid_centers]
            position_variance = np.std(x_coords) + np.std(y_coords)
            position_stability = max(0, 100 - position_variance * 2)
        else:
            position_stability = 100
        
        # Tilt stability (how consistent is shoulder level?)
        if len(valid_tilts) >= 2:
            tilt_variance = np.std(valid_tilts)
            tilt_stability = max(0, 100 - tilt_variance * 10)
        else:
            tilt_stability = 100
        
        # Lean stability (how consistent is body lean?)
        if len(valid_leans) >= 2:
            lean_variance = np.std(valid_leans)
            lean_stability = max(0, 100 - lean_variance * 5)
        else:
            lean_stability = 100
        
        return {
            'position_stability': position_stability,
            'tilt_stability': tilt_stability,
            'lean_stability': lean_stability,
            'overall_stability': (position_stability + tilt_stability + lean_stability) / 3
        }

# Test
tracker = PostureTracker(history_size=10)

# Simulate stable posture
for i in range(10):
    tracker.update(
        shoulder_center=(320 + np.random.normal(0, 2), 200 + np.random.normal(0, 2)),
        shoulder_tilt=2 + np.random.normal(0, 0.5),
        lean_angle=1 + np.random.normal(0, 0.3)
    )

stability = tracker.calculate_stability()
print("Stable Posture Test:")
print(f"  Position stability: {stability['position_stability']:.1f}")
print(f"  Tilt stability: {stability['tilt_stability']:.1f}")
print(f"  Overall: {stability['overall_stability']:.1f}")

## 5. Posture Score Calculation

In [ ]:
def calculate_posture_score(
    shoulder_tilt: float,
    lean_angle: float,
    stability: float,
    body_detected: bool = True,
    max_acceptable_tilt: float = 8.0,
    max_acceptable_lean: float = 10.0
) -> dict:
    """
    Calculate posture score for interview context.
    
    Good posture in interviews:
    - Level shoulders (not tilted)
    - Upright body (not leaning excessively)
    - Stable (not fidgeting)
    - Slightly forward lean can show engagement
    
    Returns:
        dict: Score breakdown
    """
    if not body_detected:
        return {
            'overall_score': 0,
            'body_detected': False,
            'assessment': 'no_body_detected'
        }
    
    # Shoulder level score
    tilt_deviation = abs(shoulder_tilt)
    if tilt_deviation <= 3:
        tilt_score = 100
    elif tilt_deviation <= max_acceptable_tilt:
        tilt_score = 100 - ((tilt_deviation - 3) / (max_acceptable_tilt - 3)) * 30
    else:
        tilt_score = max(30, 70 - (tilt_deviation - max_acceptable_tilt) * 5)
    
    # Body lean score
    lean_deviation = abs(lean_angle)
    if lean_deviation <= 5:
        lean_score = 100
    elif lean_deviation <= max_acceptable_lean:
        lean_score = 100 - ((lean_deviation - 5) / (max_acceptable_lean - 5)) * 30
    else:
        lean_score = max(30, 70 - (lean_deviation - max_acceptable_lean) * 3)
    
    # Stability score (already 0-100)
    stability_score = stability
    
    # Weighted average
    overall_score = (
        tilt_score * 0.25 +
        lean_score * 0.25 +
        stability_score * 0.50
    )
    
    # Determine assessment
    if overall_score >= 85:
        assessment = 'excellent'
    elif overall_score >= 70:
        assessment = 'good'
    elif overall_score >= 50:
        assessment = 'fair'
    else:
        assessment = 'needs_improvement'
    
    return {
        'overall_score': round(overall_score, 1),
        'tilt_score': round(tilt_score, 1),
        'lean_score': round(lean_score, 1),
        'stability_score': round(stability_score, 1),
        'shoulder_tilt': round(shoulder_tilt, 1),
        'lean_angle': round(lean_angle, 1),
        'is_upright': lean_deviation < 5,
        'shoulders_level': tilt_deviation < 3,
        'body_detected': True,
        'assessment': assessment
    }

# Test cases
test_cases = [
    (2, 1, 95, "Perfect upright posture"),
    (5, 3, 85, "Good posture, slight tilt"),
    (10, 15, 60, "Significant lean and tilt"),
    (3, 2, 40, "Good alignment but fidgety"),
]

print("Posture Score Tests:")
print("-" * 60)
for tilt, lean, stab, desc in test_cases:
    result = calculate_posture_score(tilt, lean, stab)
    print(f"\n{desc}")
    print(f"  Overall: {result['overall_score']}/100 ({result['assessment']})")
    print(f"  Tilt: {result['tilt_score']}, Lean: {result['lean_score']}, Stability: {result['stability_score']}")

## 6. Frame Analysis Function

In [ ]:
def analyze_frame_posture(frame, tracker=None):
    """
    Analyze a single frame for posture.
    
    Args:
        frame: BGR image
        tracker: Optional PostureTracker for temporal analysis
    
    Returns:
        dict: Posture analysis results
    """
    # Convert BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    h, w = frame.shape[:2]
    
    # Process with MediaPipe
    results = pose.process(rgb_frame)
    
    if not results.pose_landmarks:
        return {
            'body_detected': False,
            'overall_score': 0,
            'assessment': 'no_body_detected'
        }
    
    # Calculate metrics
    shoulder_metrics = calculate_shoulder_metrics(results.pose_landmarks, w, h)
    lean_metrics = calculate_body_lean(results.pose_landmarks, w, h)
    
    # Update tracker if provided
    if tracker is not None:
        tracker.update(
            shoulder_metrics['center'],
            shoulder_metrics['tilt_angle'],
            lean_metrics['lean_angle']
        )
        stability_metrics = tracker.calculate_stability()
        stability = stability_metrics['overall_stability']
    else:
        stability = 80  # Default for single frame
    
    # Calculate score
    score_result = calculate_posture_score(
        shoulder_metrics['tilt_angle'],
        lean_metrics['lean_angle'],
        stability
    )
    
    # Add additional info
    score_result['shoulder_center'] = shoulder_metrics['center']
    score_result['lean_direction'] = lean_metrics['lean_direction']
    
    return score_result

print("Frame analysis function defined")

## 7. Export Utility Functions

In [ ]:
# Create utils module for backend integration
utils_code = '''
"""
Posture Analysis Utilities

Provides functions for analyzing body posture including:
- Shoulder alignment detection
- Body lean calculation
- Stability tracking
- Posture scoring for interviews

Generated from notebook: 04_posture_estimation.ipynb
"""

import numpy as np
from collections import deque
from typing import Dict, Tuple, Optional

# Pose landmark indices
POSE_LANDMARKS = {
    "nose": 0,
    "left_shoulder": 11,
    "right_shoulder": 12,
    "left_hip": 23,
    "right_hip": 24,
}


def get_landmark(landmarks, idx: int, img_width: int, img_height: int) -> Tuple:
    """Get landmark coordinates."""
    lm = landmarks.landmark[idx]
    return (lm.x * img_width, lm.y * img_height, lm.z, lm.visibility)


def calculate_shoulder_metrics(landmarks, img_width: int, img_height: int) -> Dict:
    """Calculate shoulder alignment metrics."""
    left = get_landmark(landmarks, 11, img_width, img_height)
    right = get_landmark(landmarks, 12, img_width, img_height)
    
    level_diff = abs(left[1] - right[1])
    width = abs(left[0] - right[0])
    tilt_angle = np.degrees(np.arctan(level_diff / width)) if width > 0 else 0
    
    return {
        "center": ((left[0] + right[0]) / 2, (left[1] + right[1]) / 2),
        "width": width,
        "tilt_angle": tilt_angle,
        "is_level": tilt_angle < 5
    }


def calculate_body_lean(landmarks, img_width: int, img_height: int) -> Dict:
    """Calculate body lean from shoulder and hip alignment."""
    left_shoulder = get_landmark(landmarks, 11, img_width, img_height)
    right_shoulder = get_landmark(landmarks, 12, img_width, img_height)
    left_hip = get_landmark(landmarks, 23, img_width, img_height)
    right_hip = get_landmark(landmarks, 24, img_width, img_height)
    
    shoulder_center_x = (left_shoulder[0] + right_shoulder[0]) / 2
    hip_center_x = (left_hip[0] + right_hip[0]) / 2
    vertical_distance = abs((left_hip[1] + right_hip[1]) / 2 - (left_shoulder[1] + right_shoulder[1]) / 2)
    
    lean_angle = np.degrees(np.arctan((shoulder_center_x - hip_center_x) / vertical_distance)) if vertical_distance > 0 else 0
    
    if abs(lean_angle) < 3:
        lean_direction = "upright"
    elif lean_angle > 0:
        lean_direction = "leaning_right"
    else:
        lean_direction = "leaning_left"
    
    return {"lean_angle": lean_angle, "lean_direction": lean_direction, "is_upright": abs(lean_angle) < 5}


class PostureTracker:
    """Tracks posture over time for stability calculation."""
    
    def __init__(self, history_size: int = 30):
        self.shoulder_center_history = deque(maxlen=history_size)
        self.shoulder_tilt_history = deque(maxlen=history_size)
        self.lean_angle_history = deque(maxlen=history_size)
    
    def update(self, shoulder_center, shoulder_tilt, lean_angle):
        self.shoulder_center_history.append(shoulder_center)
        self.shoulder_tilt_history.append(shoulder_tilt)
        self.lean_angle_history.append(lean_angle)
    
    def calculate_stability(self) -> Dict:
        valid_centers = [p for p in self.shoulder_center_history if p is not None]
        valid_tilts = [t for t in self.shoulder_tilt_history if t is not None]
        
        if len(valid_centers) >= 2:
            position_variance = np.std([c[0] for c in valid_centers]) + np.std([c[1] for c in valid_centers])
            position_stability = max(0, 100 - position_variance * 2)
        else:
            position_stability = 100
        
        tilt_stability = max(0, 100 - np.std(valid_tilts) * 10) if len(valid_tilts) >= 2 else 100
        
        return {
            "position_stability": position_stability,
            "tilt_stability": tilt_stability,
            "overall_stability": (position_stability + tilt_stability) / 2
        }


def calculate_posture_score(
    shoulder_tilt: float,
    lean_angle: float,
    stability: float,
    body_detected: bool = True
) -> Dict:
    """Calculate posture score for interview context."""
    if not body_detected:
        return {"overall_score": 0, "body_detected": False, "assessment": "no_body_detected"}
    
    tilt_dev = abs(shoulder_tilt)
    tilt_score = 100 if tilt_dev <= 3 else max(30, 100 - tilt_dev * 5)
    
    lean_dev = abs(lean_angle)
    lean_score = 100 if lean_dev <= 5 else max(30, 100 - lean_dev * 3)
    
    overall_score = tilt_score * 0.25 + lean_score * 0.25 + stability * 0.50
    
    return {
        "overall_score": round(overall_score, 1),
        "tilt_score": round(tilt_score, 1),
        "lean_score": round(lean_score, 1),
        "stability_score": round(stability, 1),
        "shoulder_tilt": round(shoulder_tilt, 1),
        "lean_angle": round(lean_angle, 1),
        "body_detected": True
    }
'''

# Save to training directory
utils_path = PROJECT_ROOT / 'ml' / 'softskills' / 'training' / 'posture_utils.py'
utils_path.parent.mkdir(parents=True, exist_ok=True)

with open(utils_path, 'w') as f:
    f.write(utils_code)

print(f"Exported utilities to: {utils_path}")

## 8. Summary

### Key Findings
1. MediaPipe Pose provides 33 landmarks for full body
2. Upper body (shoulders, hips) is key for interview posture
3. Stability (lack of fidgeting) is weighted 50% of score
4. Level shoulders and upright posture indicate confidence

### Scoring Criteria
- **Shoulder Level (25%)**: Shoulders should be even
- **Body Lean (25%)**: Should be mostly upright
- **Stability (50%)**: Minimal fidgeting/movement

### Thresholds
- Shoulder tilt: <3° excellent, <8° acceptable
- Body lean: <5° upright, <10° acceptable

### Next Steps
1. Add forward/backward lean detection
2. Detect head position relative to body
3. Integrate with backend `posture_analyzer.py` service